In [1]:
from css_utility_expansion_dev import *

[NbConvertApp] Converting notebook css_utility_expansion_dev.ipynb to script
[NbConvertApp] Writing 98177 bytes to css_utility_expansion_dev.py


In [52]:
def save_TSS_by_loc_IHEC(whole_gene_file, input_path="./",output_path="./",file_name="upNkdownNk", up_num=2000, down_num=4000, unit=200):
    """
    extract TSS region by location estimation. 
    input: (1) whole_gene_file: the raw gene bed file (e.g. RefSeq.WholeGene.bed)
           (2) input_path: pickled df per cell
    output: save tss_by_loc_css_unit_all at the output path
    """
    file_lst=os.listdir(input_path)
    all_files=[os.path.join(input_path,file) for file in file_lst]
    for file in all_files:
        cell_num=file.split("/")[-1][:14]
        # if cell_num=="IHECRE00000002": break  # for test 
        with open(file,"rb") as f:
            df_pickled=pickle.load(f)
        # align the gene file and the df file according to their availability(some cells does not have chr Y)
        new_gene_lst_all, trimmed_df=remove_chrM_and_trim_gene_file_accordingly(whole_gene_file,df_pickled)
        css_lst_chr = df2longcss(trimmed_df) # list of long css per chromosome
        total_chr = len(new_gene_lst_all)       
        tss_by_loc_css_all = []
        for i in range(total_chr):
            gene_start_lst = new_gene_lst_all[i]["TxStart"]
            css_lst = css_lst_chr[i]
            tss_by_loc_css_chr = []
            for j in range(len(gene_start_lst)):
                gene_start = gene_start_lst[j]
                win_start = max(0, gene_start - up_num)  # use max to prevent negative index
                win_end = min(len(css_lst), gene_start + down_num)  # use min to prevent index out of range
                tss_by_loc_css = css_lst[win_start:win_end]
                tss_by_loc_css_chr.append(tss_by_loc_css)               
            tss_by_loc_css_all.append(tss_by_loc_css_chr)
        tss_by_loc_css_unit_all=Convert2unitCSS_main_new(tss_by_loc_css_all, unit=unit)  
        tss_by_loc_css_unit_all_flat=flatLst(tss_by_loc_css_unit_all)
        output_file_name=os.path.join(output_path,cell_num+"_prom_"+file_name+".pkl")
        with open(output_file_name,"wb") as g:
            pickle.dump(tss_by_loc_css_unit_all_flat,g)

    return print("All done!") #tss_by_loc_css_unit_all_flat

In [54]:
save_TSS_by_loc_IHEC(whole_gene_file="../database/RefSeq/hg38/RefSeq.WholeGene.bed", 
                input_path="../database/df_pickled/",
                output_path="../database/prom_IHEC/prom_css_pickled/up2kdown4k",
                file_name="up2kdown4k", 
                up_num=2000, down_num=4000, unit=200)

Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene file ...
Done!
Extracting the gene 

In [55]:
path="../database/prom_IHEC/prom_css_pickled/up2kdown4k/IHECRE00000002_prom_up2kdown4k.pkl"
with open(path,"rb") as f:
    test_css=pickle.load(f)

In [72]:
len(test_css)

21237

In [57]:
type(test_css[0])

str

In [68]:
test_css[120]

'RRRRRRRRRQQQQQQQQQQQQQQQQQQQQQ'

In [69]:
path_g="../database/css_unit_pickled/IHECRE00000001_unitcss.pkl"
with open(path_g,"rb") as g:
    test_css_g=pickle.load(g)

In [71]:
len(test_css_g)

23

In [81]:
################### This is a revised version to handle the list of list  (what we used for whole genome area) 
################### And modified to handle the faltten list (for promoter area) and an empty string element
################### When replace the original one, remain the original as commented 
def save_css_by_cell_wo_continuous_18state(path_to_css_unit_pickled, output_file, k=4, max_tokens=510):
    """
    Preprocesses CSS data and saves all processed content into a single text file, 
    removing continuous R states and splitting excess tokens into new lines.

    Args:
        path_to_css_unit_pickled (str): Path to the pickled CSS unit files.
        output_file (str): Path to save the consolidated processed file.
        k (int): Length of the k-mer. Default is 4.
        max_tokens (int): Maximum number of tokens per line. Default is 510.
    """
    # Read files from css_unit_pickled
    files = os.listdir(path_to_css_unit_pickled)
    file_path_lst = [os.path.join(path_to_css_unit_pickled, file) for file in files]
    
    all_css_kmer = []  # Collect all processed data here
    
    for file_path in file_path_lst:
        file_name = os.path.basename(file_path)
        if file_name[:4] == 'IHEC':
            file_id = file_name[:14]
        else:
            continue
        
        with open(file_path, "rb") as f:
            css = pickle.load(f)
        
        # Handle flat list input
        if isinstance(css[0], str):  # If `css` is a flat list
            css = [css]  # Wrap into a single list to process as one "chromosome"

        for css_chr_list in css:  # Iterate over sublists
            for css_chr in css_chr_list:  # Iterate over strings
                if not css_chr.strip():  # Skip empty strings
                    continue
                css_chr_kmer = seq2kmer(css_chr, k)
                target_to_remove = "R" * k  # Get rid of the word with continuous R state
                css_chr_kmer_trim = css_chr_kmer.replace(target_to_remove, "")
                # Clean up extra spaces
                css_chr_kmer_trim = ' '.join(css_chr_kmer_trim.split())
                # Split into chunks of max_tokens
                tokens = css_chr_kmer_trim.split()
                for i in range(0, len(tokens), max_tokens):
                    chunk = tokens[i:i + max_tokens]
                    all_css_kmer.append(' '.join(chunk))
    
    # Save all processed data into a single text file
    with open(output_file, "w") as g:
        for line in all_css_kmer:
            g.write(line + "\n")  # Write each chunk as a line in the file

    print("All CSS data saved into a single file: ", output_file)
    
    return


In [82]:
# to save the promoter regions only
save_css_by_cell_wo_continuous_18state(path_to_css_unit_pickled="../database/prom_IHEC/prom_css_pickled/up2kdown4k", 
                                       output_file="../database/prom_IHEC/prom_kmer_wo_cont_R/up2kdown4k/promoter_ihec_all_4mer_wo_4R.txt", 
                                       k=4, 
                                       max_tokens=510)

All CSS data saved into a single file:  ../database/prom_IHEC/prom_kmer_wo_cont_R/up2kdown4k/promoter_ihec_all_4mer_wo_4R.txt


Trimmed and chunked CSS by cell saved as .txt:  IHECRE00000001


In [27]:
file_path

'../database/pretrain/IHEC_pretrain_genome_cell01_wo4R.txt'

In [34]:
file_path_cut="../database/IHECRE00000001_test_4R_state.txt"

In [35]:
with open(file_path_cut, "r") as file:
    for line_number, line in enumerate(file, start=1):
        token_count = len(line.strip().split())  # Count the number of space-separated tokens
        print(f"Line {line_number}: {token_count} tokens")

Line 1: 510 tokens
Line 2: 510 tokens
Line 3: 510 tokens
Line 4: 510 tokens
Line 5: 510 tokens
Line 6: 510 tokens
Line 7: 510 tokens
Line 8: 510 tokens
Line 9: 510 tokens
Line 10: 510 tokens
Line 11: 510 tokens
Line 12: 510 tokens
Line 13: 510 tokens
Line 14: 510 tokens
Line 15: 510 tokens
Line 16: 510 tokens
Line 17: 510 tokens
Line 18: 510 tokens
Line 19: 510 tokens
Line 20: 510 tokens
Line 21: 510 tokens
Line 22: 510 tokens
Line 23: 510 tokens
Line 24: 510 tokens
Line 25: 510 tokens
Line 26: 510 tokens
Line 27: 510 tokens
Line 28: 510 tokens
Line 29: 510 tokens
Line 30: 510 tokens
Line 31: 510 tokens
Line 32: 510 tokens
Line 33: 510 tokens
Line 34: 510 tokens
Line 35: 510 tokens
Line 36: 510 tokens
Line 37: 510 tokens
Line 38: 510 tokens
Line 39: 510 tokens
Line 40: 510 tokens
Line 41: 510 tokens
Line 42: 510 tokens
Line 43: 510 tokens
Line 44: 510 tokens
Line 45: 510 tokens
Line 46: 510 tokens
Line 47: 510 tokens
Line 48: 510 tokens
Line 49: 510 tokens
Line 50: 510 tokens
Line 51: 

In [12]:
df_path="../database/df_pickled/IHECRE00000001.4_df_pickled.pkl"
with open(df_path,"rb") as f:
    test_df=pickle.load(f)

In [13]:
test_df

,chromosome,start,end,state,length,unit,state_seq,state_seq_full
0,chr10,0,54200,18,54200,271,R,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR...
1,chr10,54200,55800,17,1600,8,Q,QQQQQQQQ
2,chr10,55800,73600,18,17800,89,R,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR...
3,chr10,73600,74200,2,600,3,B,BBB
4,chr10,74200,86000,17,11800,59,Q,QQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ...
...,...,...,...,...,...,...,...,...
595033,chrX,155997800,155998200,13,400,2,M,MM
595034,chrX,155998200,156004800,12,6600,33,L,LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL
595035,chrX,156004800,156005000,13,200,1,M,M
595036,chrX,156005000,156006000,12,1000,5,L,LLLLL


In [14]:
test_css=df2unitcss(test_df)

In [15]:
type(test_css)

list

In [16]:
len(test_css)

23

In [11]:
len(test_css[0])

668987

In [10]:
test_css[0][5000:5400]

'EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEFFFFFFFFFRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRIIFEEEEEEEFFFFFFFFFFFFFFFFFEEEGGGGGGGGGGGGGGGGGGGGFFFFFFBAAAAAABBBBBBBDKKKKKGGGGGGGGGGGGKKKKKKKBBAAAABBBBKKKKKKKKKKDDKFFFGGGGGFEEEEEEEEEEEEEEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFKKFFFFEEEEEEEEEEEEEEEEEEEEEEEFFFFFFFFFEEEEEEEE'

In [ ]:
df_path="../database/df_pickled"
output_path="../database/css_unit_pickled"
for file_name in os.listdir(df_path):
    with open(file_name,"rb") as f:
        df=pickle.load(f)
        if file_name[:4] == 'IHEC':
            file_id = file_name[:14]
        else:
            pass
        if file_id[12:14]=="04":
            break
        unit_css=df2unitcss(df)
        output_file_name=os.path.join(output_path,file_id+"_unitcss.pkl") 
        with open(output_file_name,"wb") as g:
            pickle.dump(unit_css,g)


FileNotFoundError: [Errno 2] No such file or directory: 'IHECRE00000002.3_df_pickled.pkl'

In [18]:
df_path="../database/df_pickled"
for file_name in os.listdir(df_path):
    print(file_name[:14])

IHECRE00000001
IHECRE00000002
IHECRE00000004
IHECRE00000008
IHECRE00000009
IHECRE00000010
IHECRE00000011
IHECRE00000013
IHECRE00000014
IHECRE00000016
IHECRE00000017
IHECRE00000019
IHECRE00000022
IHECRE00000024
IHECRE00000026
IHECRE00000027
IHECRE00000031
IHECRE00000034
IHECRE00000035
IHECRE00000041
IHECRE00000043
IHECRE00000045
IHECRE00000048
IHECRE00000049
IHECRE00000051
IHECRE00000053
IHECRE00000054
IHECRE00000055
IHECRE00000056
IHECRE00000058
IHECRE00000059
IHECRE00000060
IHECRE00000070
IHECRE00000071
IHECRE00000073
IHECRE00000075
IHECRE00000076
IHECRE00000083
IHECRE00000084
IHECRE00000085
IHECRE00000087
IHECRE00000090
IHECRE00000094
IHECRE00000095
IHECRE00000096
IHECRE00000097
IHECRE00000099
IHECRE00000101
IHECRE00000102
IHECRE00000105
IHECRE00000108
IHECRE00000109
IHECRE00000112
IHECRE00000113
IHECRE00000114
IHECRE00000117
IHECRE00000118
IHECRE00000121
IHECRE00000123
IHECRE00000124
IHECRE00000125
IHECRE00000126
IHECRE00000128
IHECRE00000129
IHECRE00000132
IHECRE00000133
IHECRE0000

In [22]:
# #### To save css_unit from df_pickled in bulk
# df_path = "../database/df_pickled"
# output_path = "../database/css_unit_pickled"

# for file_name in os.listdir(df_path):
#     file_path = os.path.join(df_path, file_name)  # Full path to the file
#     with open(file_path, "rb") as f:
#         df = pickle.load(f)
        
#         # Extract file_id only for files starting with 'IHEC'
#         if file_name[:4] == 'IHEC':
#             file_id = file_name[:14]
#         else:
#             continue  # Skip files that do not match the condition

#         # Process the dataframe and save the output
#         unit_css = df2unitcss(df)
#         output_file_name = os.path.join(output_path, f"{file_id}_unitcss.pkl")
#         with open(output_file_name, "wb") as g:
#             pickle.dump(unit_css, g)
# ## this has been conducted, no need to execute again

In [ ]:
css_unit_path="../database/roadmap/prom/up2kdown4k/all_genes/"
output_path="../database/final_test/"
prom_css_Kmer_by_cell(path=path, output_path=output_path, k=4)

In [ ]:
##########################
##########################
##########################
##########################
########################## Corrected verion for max token numbers

def save_css_by_cell_wo_continuous_18state(path_to_css_unit_pickled, output_path, k=4, max_tokens=510):
    """
    Preprocesses and saves CSS per cell, per chromosome, removing continuous R states 
    and splitting excess tokens into new lines, saving output as .txt.

    Args:
        path_to_css_unit_pickled (str): Path to the pickled CSS unit files.
        output_path (str): Path to save the processed files.
        k (int): Length of the k-mer. Default is 4.
        max_tokens (int): Maximum number of tokens per line. Default is 510.
    """
    # Read files from css_unit_pickled
    files = os.listdir(path_to_css_unit_pickled)
    file_path_lst = [os.path.join(path_to_css_unit_pickled, file) for file in files]
    
    for file_path in file_path_lst:
        file_name = os.path.basename(file_path)
        if file_name[:4] == 'IHEC':
            file_id = file_name[:14]
        else:
            continue
        
        with open(file_path, "rb") as f:
            css = pickle.load(f)
        
        css_kmer = []
        for css_chr in css:
            css_chr_kmer = seq2kmer(css_chr, k)
            target_to_remove = "R" * k  # Get rid of the word with continuous R state
            css_chr_kmer_trim = css_chr_kmer.replace(target_to_remove, "")
            # Clean up extra spaces
            css_chr_kmer_trim = ' '.join(css_chr_kmer_trim.split())
            # Split into chunks of max_tokens
            tokens = css_chr_kmer_trim.split()
            for i in range(0, len(tokens), max_tokens):
                chunk = tokens[i:i + max_tokens]
                css_kmer.append(' '.join(chunk))
        
        # Save output as .txt file
        output_file_name = os.path.join(output_path, file_id + "_test_" + str(k) + "R_state.txt")
        with open(output_file_name, "w") as g:
            for line in css_kmer:
                g.write(line + "\n")  # Write each chunk as a line in the file

        print("Trimmed and chunked CSS by cell saved as .txt: ", file_id)
    
    return



In [31]:
css_unit_path="../database/css_unit_pickled_test/"
wo_R_output_path="../database"
save_css_by_cell_wo_continuous_18state(css_unit_path, wo_R_output_path, k=4, max_tokens=510)

Trimmed and chunked CSS by cell saved as .txt:  IHECRE00000001


In [41]:
###################### if successful, do this to prepare pretrain IHEC all

def save_and_concatenate_css(path_to_css_unit_pickled, concatenated_output_file, k=4, max_tokens=510):
    """
    Preprocesses CSS per cell, per chromosome, removing continuous R states,
    splitting excess tokens into new lines, and saving all processed CSS into a single file.

    Args:
        path_to_css_unit_pickled (str): Path to the pickled CSS unit files.
        concatenated_output_file (str): Path to save the concatenated .txt file.
        k (int): Length of the k-mer. Default is 4.
        max_tokens (int): Maximum number of tokens per line. Default is 510.
    """
    # Read files from css_unit_pickled
    files = os.listdir(path_to_css_unit_pickled)
    file_path_lst = [os.path.join(path_to_css_unit_pickled, file) for file in files]

    all_kmers = []  # List to store all processed css_kmers for concatenation

    for file_path in file_path_lst:
        file_name = os.path.basename(file_path)
        if file_name[:4] == 'IHEC':
            file_id = file_name[:14]
        else:
            continue
        
        with open(file_path, "rb") as f:
            css = pickle.load(f)
        
        for css_chr in css:
            css_chr_kmer = seq2kmer(css_chr, k)
            target_to_remove = "R" * k  # Get rid of the word with continuous R state
            css_chr_kmer_trim = css_chr_kmer.replace(target_to_remove, "")
            # Clean up extra spaces
            css_chr_kmer_trim = ' '.join(css_chr_kmer_trim.split())
            # Split into chunks of max_tokens
            tokens = css_chr_kmer_trim.split()
            for i in range(0, len(tokens), max_tokens):
                chunk = tokens[i:i + max_tokens]
                all_kmers.append(' '.join(chunk))  # Add each chunk directly to all_kmers

    # Save concatenated output to a single .txt file
    with open(concatenated_output_file, "w") as concat_file:
        for line in all_kmers:
            concat_file.write(line + "\n")

    print(f"All processed CSS concatenated and saved to {concatenated_output_file}")



In [42]:
save_and_concatenate_css(
    path_to_css_unit_pickled="../database/css_unit_pickled",
    concatenated_output_file="../database/pretrain/IHEC_all_genome_cut_4mer_wo_R_all_concatenated_css.txt",
    k=4,
    max_tokens=510
)

All processed CSS concatenated and saved to ../database/pretrain/IHEC_all_genome_cut_4mer_wo_R_all_concatenated_css.txt


In [25]:
# # [IHEC] Pretrain data preprocessing and storing (for 18-state version)

# # Preprocessing for removing continuous R state for pretrain dataset
# # 1-1. Save the CSS per cell, per chromosome
# def save_css_by_cell_wo_continuous_18state(path_to_css_unit_pickled, output_path,k=4):
#     # read files from css_unit_pickled
#     files=os.listdir(path_to_css_unit_pickled)
#     file_path_lst=[os.path.join(path_to_css_unit_pickled,file) for file in files]
#     for file_path in file_path_lst:
#         file_name=os.path.basename(file_path)
#         if file_name[:4] == 'IHEC':
#             file_id = file_name[:14]
#         else:
#             pass
#         # ##########################
#         # if str(file_id)=="E003":
#         #     break  # for test
#         # ##########################
#         with open(file_path,"rb") as f:
#             css=pickle.load(f)
#         css_kmer=[]
#         for css_chr in css:
#             css_chr_kmer=seq2kmer(css_chr,k)
#             target_to_remove="R"*k   # get rid of the word with continuous 15th state "o"
#             css_chr_kmer_trim = css_chr_kmer.replace(target_to_remove, "")
#             # clean up extra spaces
#             css_chr_kmer_trim = ' '.join(css_chr_kmer_trim.split())
#             css_kmer.append(css_chr_kmer_trim)
#         output_file_name=os.path.join(output_path,file_id+"_unitcss_wo_all"+str(k)+"R_state.pkl")    
#         with open(output_file_name, "wb") as g:
#             pickle.dump(css_kmer, g)  # note that it is chromosome-wise list (each element corresponds to each chromosome)

#         print("trimmed css by cell saved: ",file_id)
#     return 

In [2]:
# css_unit_path="../database/css_unit_pickled"
# wo_R_output_path="../database/wo_R_css_unit/4mer"

# save_css_by_cell_wo_continuous_18state(css_unit_path,wo_R_output_path, k=4)

In [27]:
with open("../database/wo_R_css_unit/4mer/IHECRE00000001_unitcss_wo_all4R_state.pkl", "rb") as fff:
    test_wor=pickle.load(fff)

In [28]:
type(test_wor)

list

In [30]:
len(test_wor[0])

1946549

In [31]:
test_wor[0][:400]

'RRRQ RRQQ RQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQR QQRR QRRR RRRB RRBB RBBB BBBQ BBQQ BQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQQ QQQR QQRR QRRR RRRM RRMM RMMM MMMM '

In [36]:
# Preprocessing for removing continuous O state for pretrain dataset
# 2-2. Concatenate all the cells and create one .txt file (IHEC file version)
# (Note. new line joining chromosome-wise and cell-wise)
# def kmerCSS_to_pretrain_data_ihec(path_to_kmer_css_unit_pickled,output_path):
#     files=os.listdir(path_to_kmer_css_unit_pickled)
#     file_path_lst=[os.path.join(path_to_kmer_css_unit_pickled,file) for file in files]

#     css_all=[]
#     for file_path in file_path_lst:
#         file_name=os.path.basename(file_path)
#         if file_name[:4] == 'IHEC':
#             file_id = file_name[:14]
#         else:
#             pass
#         # ##########################
#         # if str(file_id)=="E003":
#         #     break  # for test
#         # # ##########################
#         with open(file_path,"rb") as f:
#             css=pickle.load(f)

#         css_per_cell='\n'.join(css)   # join the chromosome by new line

#         css_all.append(css_per_cell)   

#     css_all_cell='\n'.join(css_all)  # join the cell by new line

#     output_name=os.path.join(output_path,"IHEC_pretrain_genome_all_wo4R.txt") 
#     with open(output_name, "w") as g:
#         g.write(css_all_cell)


def kmerCSS_to_pretrain_data_ihec(path_to_kmer_css_unit_pickled, output_path):
    files = os.listdir(path_to_kmer_css_unit_pickled)
    file_path_lst = [os.path.join(path_to_kmer_css_unit_pickled, file) for file in files]

    output_name = os.path.join(output_path, "IHEC_pretrain_genome_all_wo4R.txt")
    with open(output_name, "w") as g:
        for file_path in file_path_lst:
            file_name = os.path.basename(file_path)
            
            if file_name[:4] == 'IHEC':
                file_id = file_name[:14]
            else:
                continue
            
            # Load the CSS data for each cell
            with open(file_path, "rb") as f:
                css = pickle.load(f)
            
            # Write the chromosome data for this cell, joined by new lines, directly to the output file
            css_per_cell = '\n'.join(css)  # Join chromosomes by new lines
            g.write(css_per_cell + '\n')   # Write to the output file and add a newline between cells




In [37]:
path_to_kmer_css_unit_pickled="../database/wo_R_css_unit/4mer"
output_path="../database/pretrain"
kmerCSS_to_pretrain_data_ihec(path_to_kmer_css_unit_pickled,output_path)

In [4]:
file_path="../database/wo_R_css_unit/4mer/IHECRE00000001_unitcss_wo_all4R_state.pkl"
output_path="../database/pretrain"
output_name = os.path.join(output_path, "IHEC_pretrain_genome_cell01_wo4R.txt")

with open(file_path, "rb") as f:
    css = pickle.load(f)
    with open(output_name, "w") as g:
        css_per_cell = '\n'.join(css) # Join chromosomes by new lines
        g.write(css_per_cell + '\n')

In [5]:
output_name

'../database/pretrain/IHEC_pretrain_genome_cell01_wo4R.txt'

In [12]:
file_path = "../database/pretrain/IHEC_pretrain_genome_cell01_wo4R.txt"
output_path = "../database/pretrain/invalid_lines.txt"

try:
    with open(file_path, "r") as f:
        lines = f.readlines()

    invalid_lines = []
    valid_chars = set("ABCDEFGHIJKLMNOPQR")

    for i, line in enumerate(lines):
        line = line.strip()
        # Split by spaces and validate each 4-mer
        tokens = line.split()
        if not all(len(token) == 4 and set(token).issubset(valid_chars) for token in tokens):
            invalid_lines.append((i + 1, line))

    # Save invalid lines to a file
    with open(output_path, "w") as out_file:
        for line_number, content in invalid_lines:
            out_file.write(f"Line {line_number}: {content}\n")

    print(f"Found {len(invalid_lines)} invalid lines. Saved to {output_path}")

except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Found 0 invalid lines. Saved to ../database/pretrain/invalid_lines.txt


In [19]:
from transformers import DNATokenizer

# Initialize the tokenizer
tokenizer = DNATokenizer.from_pretrained("dev/training/src/transformers/dnabert-config/bert-config-4")

# Test a space-separated input
test_line = "QQQQ QQQQ QQQQ QQQQ"

try:
    batch_outputs = tokenizer.batch_encode_plus(
        [test_line],
        add_special_tokens=True,
        max_length=512,
        truncation=True
    )
    print("Encoded outputs:", batch_outputs)
except Exception as e:
    print(f"Tokenizer failed: {e}")


ImportError: cannot import name 'DNATokenizer' from 'transformers' (/home/lee/miniconda3/envs/chrombert/lib/python3.8/site-packages/transformers/__init__.py)

In [ ]:
# place this at the end

# if __name__ == "__main__":
#     if len(sys.argv) > 1:
#         func_name = sys.argv[1]
#         globals()[func_name]()  

In [2]:
# test_file.bed is the 4th column (state is converted to only have number)
# using the command: awk 'BEGIN{FS=OFS="\t"} {split($4, a, "_"); $4 = a[1]; print}' filename.bed > modified_filename.bed

bed_path="../database/bed/IHEC_unzipped/test_file.bed"

In [3]:
def bed2df_expanded(filename, state_num=15):
    """Create an expanded dataframe from the .bed file.
    Dataframe contains following columns:
    chromosome |  start |  end  | state | length | unit | state_seq | state_seq_full"""
    if not os.path.exists(filename):
        raise FileNotFoundError("Please provide a valid file path.")

    df_raw=pd.read_csv(filename, sep='\t', lineterminator='\n', header=None, low_memory=False)
    df=df_raw.rename(columns={0:"chromosome",1:"start",2:"end",3:"state"})
    df=df[:-1]
    df["start"]=pd.to_numeric(df["start"])
    df["end"]=pd.to_numeric(df["end"])
    df["state"]=pd.to_numeric(df["state"])
    df["length"]=df["end"]-df["start"]
    df["unit"]=(df["length"]/200).astype(int)  # chromatin state is annotated every 200 bp (18th May 2022)
    ##################### modified to accommodate 18 states
    if state_num==18:
        df["state_seq"]=df["state"].map(state_dict_18)
        df["state_seq_full"]=df["unit"]*df["state_seq"]
    #######################################################
    else:
        df["state_seq"]=df["state"].map(state_dict)
        df["state_seq_full"]=df["unit"]*df["state_seq"]
    
    return df 

In [4]:
df=bed2df_expanded(bed_path, 18)

In [5]:
df

,chromosome,start,end,state,length,unit,state_seq,state_seq_full
0,chr10,0,73800,18,73800,369,R,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR...
1,chr10,73800,74200,2,400,2,B,BB
2,chr10,74200,76200,6,2000,10,F,FFFFFFFFFF
3,chr10,76200,76600,2,400,2,B,BB
4,chr10,76600,87200,18,10600,53,R,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR...
...,...,...,...,...,...,...,...,...
331556,chrX,156000200,156000400,11,200,1,K,K
331557,chrX,156000400,156000600,9,200,1,I,I
331558,chrX,156000600,156001200,10,600,3,J,JJJ
331559,chrX,156001200,156002000,9,800,4,I,IIII


In [6]:
df["chromosome"].unique()

array(['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16',
       'chr17', 'chr18', 'chr19', 'chr1', 'chr20', 'chr21', 'chr22',
       'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
       'chrX'], dtype=object)

In [8]:
np.size(df["chromosome"].unique())

23

In [20]:
test_css=df2unitcss(df)

In [22]:
len(test_css[0])

668987

In [26]:
test_css[0][5000:5500]

'EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEEEEEEEEEEEEEEEEEEEEEEEEEEEEFFJJJJJJJJJFFFFFFFFFFFFFFFAAAAAAAAAAJJJGGGGGFFFFEEEEEEEEEEFGHCCAAAAAAAAAAAAAIIIIFFFFFFFKKKKKIIIIIIIIIIIIIIIIJJJJFFFFFFFFJJJJJJJJJIIJJJJJJJJJIIIIIJJFFFFFEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEGGGGGGGGGGGGGGGEEEEEEEEEEEEEEFFFFEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE'

In [9]:
unit_css_test=df2unitcss(df)

In [13]:
len(unit_css_test[0])

668987

In [12]:
type(unit_css_test)

list

In [17]:
with open("../database/cell_pickle/df_cell001.pkl", "rb") as g:
    pick_g=pickle.load(g)

In [18]:
type(pick_g)

pandas.core.frame.DataFrame

In [19]:
pick_g

,chromosome,start,end,state,length,unit,state_seq,state_seq_full
0,chr1,0,9800,15,9800,98,O,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,chr1,9800,10600,9,800,8,I,IIIIIIII
2,chr1,10600,540400,15,529800,5298,O,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3,chr1,540400,540800,1,400,4,A,AAAA
4,chr1,540800,569800,15,29000,290,O,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
...,...,...,...,...,...,...,...,...
510145,chrY,59003800,59005800,15,2000,20,O,OOOOOOOOOOOOOOOOOOOO
510146,chrY,59005800,59006000,9,200,2,I,II
510147,chrY,59006000,59011800,15,5800,58,O,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
510148,chrY,59011800,59026000,9,14200,142,I,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...


In [20]:
with open("../database/roadmap/prom/up2kdown4k/all_genes/E001_prom_up2kdown4k.pkl", "rb") as f:
    pick_f=pickle.load(f)

In [22]:
len(pick_f)

24

In [2]:
with open("../database/IHECRE00002309.1_df_pickled.pkl", "rb") as ff:
    pick_f=pickle.load(ff)

In [3]:
pick_f

,chromosome,start,end,state,length,unit,state_seq,state_seq_full
0,chr10,0,73800,18,73800,369,R,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR...
1,chr10,73800,74200,2,400,2,B,BB
2,chr10,74200,74400,11,200,1,K,K
3,chr10,74400,75800,17,1400,7,Q,QQQQQQQ
4,chr10,75800,76000,15,200,1,O,O
...,...,...,...,...,...,...,...,...
379399,chrX,155963000,155963800,16,800,4,P,PPPP
379400,chrX,155963800,155964400,12,600,3,L,LLL
379401,chrX,155964400,155968800,13,4400,22,M,MMMMMMMMMMMMMMMMMMMMMM
379402,chrX,155968800,155969800,12,1000,5,L,LLLLL


In [27]:
type(ff)

_io.BufferedReader